In [1]:
import xml.etree.ElementTree
from fileoperations.fileoperations import get_filenames_in_dir
from fileoperations.slugify_tr import slugify_tr

In [2]:
owl_files = get_filenames_in_dir('../owl/', keyword='*.owl')[0]

In [3]:
change_dict = {}
for of in owl_files:
    e = xml.etree.ElementTree.parse(of).getroot()
    for anno in e.findall('{http://www.w3.org/2002/07/owl#}AnnotationAssertion'):
        try:
            iri = anno.find('{http://www.w3.org/2002/07/owl#}IRI').text
            if 'OWL' in iri:
                iri = iri.split('#')[1]
                prop_attr = anno.find('{http://www.w3.org/2002/07/owl#}AnnotationProperty').attrib
                literal = anno.find('{http://www.w3.org/2002/07/owl#}Literal')
                literal_str = literal.text

                try:
                    if 'abbreviatedIRI' in prop_attr.keys():
                        if prop_attr['abbreviatedIRI'] == 'rdfs:label':
                            lang = literal.attrib['{http://www.w3.org/XML/1998/namespace}lang']
                            if lang == 'en':
                                if iri in change_dict.keys():
                                    assert change_dict[iri] == literal_str
                                else:
                                    change_dict[iri] = slugify_tr(literal_str)
                except AssertionError:  # iri has multiple mappings
                    if change_dict[iri] is not None:
                        print iri
                        print literal_str
                        print change_dict[iri]
                        print ''
                        import pdb
                        pdb.set_trace()
                
                    #if 'OWL' in prop_attr['IRI']:
                    #    try:
                    #        if iri in change_dict.keys():
                    #            assert change_dict[iri] == literal_str
                    #    except AssertionError:  # iri has multiple mappings
                    #        if change_dict[iri] is not None:
                    #            print iri
                    #            print literal_str
                    #            print change_dict[iri]
                    #            print ''
                    #            import pdb
                    #            pdb.set_trace()
                
        except AttributeError:
            pass

ParseError: no element found: line 1, column 0

In [ ]:
import io
for of in owl_files:
    # Read in the file
    with open(of, 'r') as file :
        filedata = unicode(file.read(), 'utf8')
        
    # Replace the target string
    for key, val in change_dict.items():
        filedata = filedata.replace(key, val)

    filedata.encode('utf8', 'replace')
    
    # Write the file out again
    with io.open(of, 'w', encoding='utf8') as file:
        file.write(filedata)